# Theory

Added more equations to fill up the intermediate steps of the ICA derivation from the lecture note.

**Symbols**:

* $S$: $n \times m$, source audio, the one we try to recover with ICA. $n$: number of sources/microphones; $m$: number of time steps recorded.
* $A$: $n \times n$, mixing matrix, mixing sources in a linear fashion.
* $X$: $n \times m$, actual recorded audio after mixing
* $W$: $n \times n$, unmixing matrix, the one we try to learn from data $X$.

**Relationships**:

\begin{align*}
X &= A S\\
S & = W X \\
W &= A^{-1} \\
\end{align*}


**Convention**: 

* $w_j^T$ mean the $j$th row of $W$;
* $x^{(i)}$ means the $i$th column of $X$, i.e. the $i$th sample;
* $s_j^{(i)}$ means the element at the $j$th row and $i$th column (i.e. at time step $(i)$) of $S$.

Each source $s_j^{(i)}$ is given by a density $p_s$, then

$$ 
p(S) = \prod_{i=1}^{m} \prod_{j=1}^{n} p_s(s_j^{(i)}) 
$$

The density in terms of $X$ is (from linear algebra), which can also be interpreted as likelihood of $W$,

$$ 
p(X) = p(W^{-1}S) = \prod_{i=1}^{m} \bigg (\prod_{j=1}^{n} p_s(w_j^T x^{(i)}) \cdot \left|W\right| \bigg)  = L(X;W) 
$$

Note that $\left|W\right|$ is OUTSIDE the second $\prod$.

**Note**: *I need to understand this step better in the future*.

The log-likelihood is 

$$
\ell(X; W) = \sum_{i=1}^m \bigg( \sum_{j=1}^{n} \log p_s(w_j^T x^{(i)}) + \log \left|W\right| \bigg)
$$

To calculate the density, a reasonable assumption for its CDF is the sigmoid function, e.g. $g(a) = \frac{1}{1 + e^{-a}}$, so $p_s = g'$. Then

$$
p_s(w_j^T x^{(i)}) = g'(w_j^T x^{(i)}) 
$$

Replacing it back into $\ell(X; W)$,

$$
\ell(X; W) = \sum_{i=1}^m\sum_{j=1}^{n} \bigg(\log g'(w_j^T x^{(i)}) + \log \left|W\right| \bigg)
$$

#### Now derive the gradient of log-likelihood

For clarity, set $g'(w_j^T x^{(i)}) = g(1 - g) = g - g^2$, which is a convenient property of the sigmoid function.

For $w_j^Tx^{(i)}$,

\begin{align*}
\nabla_W \log g(1 - g) 
&= \frac{(1 - 2g) g (1- g)}{g(1 - g)} \nabla_W (w_j^T x^{(i)})  \\
&= (1 - 2g)\nabla_W (w_j^T x^{(i)})
\end{align*}

Inspect $\nabla_W (w_j^T x^{(i)})$ closely, it's a $n\times n$ matrix with all rows zero but $j$th row equaling to $x^{(i)}$.

\begin{align*}
\nabla_W (w_j^T x^{(i)}) = \begin{bmatrix}
0 & \cdots & 0 \\ 
\vdots & \ddots & \vdots \\
x_0^{(i)} & \cdots & x_n^{(i)} \\ 
\vdots & \ddots & \vdots \\
0 & \cdots & 0 \\ 
\end{bmatrix}
\end{align*}

Therefore, equivalently,

\begin{align*}
\sum_{j=1}^n (1 - 2g(w_j^T x^{(i)}))\nabla_W (w_j^T x^{(i)}) = \begin{bmatrix}
1 - 2 g(w_1^T x^{(i)}) \\ 
1 - 2 g(w_2^T x^{(i)}) \\ 
\vdots \\
1 - 2 g(w_n^T x^{(i)}) \\ 
\end{bmatrix} {x^{(i)}}^T
\end{align*}

On the other hand,

\begin{align*}
\nabla_W \log \left|W\right| = \frac{\left|W\right| (W^T)^{-1}}{\left|W\right|} = (W^T)^{-1}
\end{align*}

Combining all pieces, we get the formula for gradient ascent as presented in the lecture note.

\begin{align*}
\nabla_W \ell(X;W) = \begin{bmatrix}
1 - 2 g(w_1^T x^{(i)}) \\ 
1 - 2 g(w_2^T x^{(i)}) \\ 
\vdots \\
1 - 2 g(w_n^T x^{(i)}) \\ 
\end{bmatrix} {x^{(i)}}^T + (W^T)^{-1}
\end{align*}

**Note**: ICA only applies when the number of sources and that of microphones are the same.

# Implementation

**Heads up**: matrices in numpy are row matrices, as opposed to the column matrices in the lecture notes

In [1]:
### Independent Components Analysis
###
### This program requires a working installation of:
###
### On Mac:
###     1. portaudio: On Mac: brew install portaudio
###     2. sounddevice: pip install sounddevice
###
### On windows:
###      pip install pyaudio sounddevice
###

import sounddevice as sd
import numpy as np

Fs = 11025

def normalize(dat):
    return 0.99 * dat / np.max(np.abs(dat))

def load_data():
    mix = np.loadtxt('mix.dat')
    return mix

def play(vec):
    sd.play(vec, Fs, blocking=True)

def sigmoid(x):
    """
    A numerically stable sigmoid function for the input x.
    
    It calculates positive and negative elements with different equations to 
    prevent overflow by avoid exponentiation with large positive exponent, 
    thus achieving numerical stability.
    
    For negative elements in x, sigmoid uses this equation
    
    $$sigmoid(x_i) = \frac{e^{x_i}}{1 + e^{x_i}}$$
    
    For positive elements, it uses another equation:
    
    $$sigmoid(x_i) = \frac{1}{e^{-x_i} + 1}$$
    
    The two equations are equivalent mathematically. 
    
    x is of shape: B x H
    """
    ### YOUR CODE HERE    
    pos_mask = (x >= 0)
    neg_mask = (x < 0)

    # specify dtype! otherwise, it may all becomes zero, this could have different
    # behaviors depending on numpy version
    z = np.zeros_like(x, dtype=float)
    z[pos_mask] = np.exp(-x[pos_mask])
    z[neg_mask] = np.exp(x[neg_mask])

    top = np.ones_like(x, dtype=float)
    top[neg_mask] = z[neg_mask]
    s = top / (1 + z)
    ### END YOUR CODE
    return s

def unmixer(X):
    # M: length
    # N: number of microphones
    M, N = X.shape
    W = np.eye(N)
    losses = []

    anneal = [0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.02, 0.02, 0.01, 0.01,
              0.005, 0.005, 0.002, 0.002, 0.001, 0.001]
    print('Separating tracks ...')
    ######## Your code here ##########
    for alpha in anneal:
        print('working on alpha = {0}'.format(alpha))
        for xi in X:
            p1 = np.outer(1 - 2 * sigmoid(np.dot(W, xi.T)), xi)
            p2 = np.linalg.inv(W.T)
            W += alpha * (p1 + p2)
    ###################################
    return W

def unmix(X, W):
    ######### Your code here ##########
    S = np.dot(X, W.T)
    ##################################
    return S

In [2]:
X = normalize(load_data())

In [3]:
for i in range(X.shape[1]):
    print('Playing mixed track %d' % i)
    play(X[:, i])

Playing mixed track 0
Playing mixed track 1
Playing mixed track 2
Playing mixed track 3
Playing mixed track 4


In [4]:
# Note, W and X are both row matrices
%time W = unmixer(X)

Separating tracks ...
working on alpha = 0.1
working on alpha = 0.1
working on alpha = 0.1
working on alpha = 0.05
working on alpha = 0.05
working on alpha = 0.05
working on alpha = 0.02
working on alpha = 0.02
working on alpha = 0.01
working on alpha = 0.01
working on alpha = 0.005
working on alpha = 0.005
working on alpha = 0.002
working on alpha = 0.002
working on alpha = 0.001
working on alpha = 0.001
CPU times: user 43.6 s, sys: 781 ms, total: 44.4 s
Wall time: 44 s


In [5]:
W_expected = np.array([[ 72.15081922,  28.62441682,  25.91040458, -17.2322227 , -21.191357  ],
                       [ 13.45886116,  31.94398247,  -4.03003982, -24.0095722 , 11.89906179 ],
                       [ 18.89688784,  -7.80435173,  28.71469558,  18.14356811, -21.17474522],
                       [ -6.0119837 ,  -4.15743607,  -1.01692289,  13.87321073, -5.26252289 ],
                       [ -8.74061186,  22.55821897,   9.61289023,  14.73637074, 45.28841827 ]])

In [6]:
assert ((W - W_expected) < 1e-8).all()

In [7]:
S = normalize(unmix(X, W))

In [8]:
W.shape

(5, 5)

In [9]:
X.shape

(53442, 5)

In [10]:
S.shape

(53442, 5)

In [11]:
# Copied from http://cs229.stanford.edu/ps/ps4/q4/bellsej.m
# % now have a listen --- You should have the following five samples:
# % * Godfather
# % * Southpark
# % * Beethoven 5th
# % * Austin Powers
# % * Matrix (the movie, not the linear algebra construct :-) 

# I think the Beethoven 5th & Austin Powers are reordered!
for i in range(S.shape[1]):
    print('Playing separated track %d' % i)
    play(S[:, i])

Playing separated track 0
Playing separated track 1
Playing separated track 2
Playing separated track 3
Playing separated track 4
